In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///../Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table

# Map Measurement class
Measurement = Base.classes.measurement

# Map Station class
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Inspect Tables
inspector = inspect(engine)

In [ ]:
# Get a list of column names and types for Measurement Table
columns = inspector.get_columns('Measurement')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Get a list of column names and types for Measurement Table
columns = inspector.get_columns('Station')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Find the most recent date in the data set.
mosrec_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
mosrec_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
previous_year = (dt.date(2017, 8, 23)) - dt.timedelta(days=365)
previous_year

In [ ]:
# Perform a query to retrieve the date and precipitation scores
prevyear_prcp = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= "2016-08-23").\
    order_by(Measurement.date).all()

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
prevyear_prcp_df = pd.DataFrame(prevyear_prcp, columns=['date', 'precipitation'])
prevyear_prcp_df.set_index('date', inplace=True)

# Sort the dataframe by date
prevyear_prcp_df.sort_values('date')
prevyear_prcp_df

In [ ]:
max = len(prevyear_prcp_df['precipitation'])
max

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
prevyear_prcp_df.plot()
plt.ylabel("Inches")
plt.xlabel("Date")
plt.xticks(np.arange(0, max, step=max/9), ['2016-08-23', '2016-10-01', '2016-11-09', '2016-12-19', '2017-01-28', '2017-03-09', '2017-04-18', '2017-05-31', '2017-07-10'], rotation='90')
plt.yticks(np.arange(0, 8, step=1))
plt.tight_layout()
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prevyear_prcp_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_total = session.query(Measurement.station).distinct().count()
station_total

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
station_activity = session.query(Measurement.station, func.count(Measurement.tobs)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.tobs).desc()).all()
station_activity

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
# Create list of query functions
sel = [func.min(Measurement.tobs), 
       func.max(Measurement.tobs), 
       func.avg(Measurement.tobs)]

# query the table by unpacking list and filtering for station USC00519281
mosact_station_averages = session.query(*sel).\
    filter(Measurement.station == 'USC00519281').all()
mosact_station_averages

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station
mosact_station_tobs = session.query(Measurement.tobs).\
    filter(Measurement.station == 'USC00519281').\
    filter(Measurement.date >= "2016-08-23").all()

# Save the query results as a Pandas DataFrame
mosact_station_tobs_df = pd.DataFrame(mosact_station_tobs, columns=['tobs'])

# Plot the query results as a histogram using 'bins=12'
mosact_station_tobs_df.plot.hist(bins=12)
plt.xlabel('Temperature')
plt.tight_layout()
plt.show

# Close session

In [ ]:
# Close Session
session.close()

In [ ]:
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def start_end_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    defsel = [func.min(Measurement.tobs),
           func.avg(Measurement.tobs),
           func.max(Measurement.tobs)]
    
    return session.query(*defsel).\
        filter(Measurement.date >= start_date).\
        filter(Measurement.date <= end_date).all()

print(calc_temps('2015-2-28', '2017-03-05'))